Git
how to choose companies
Time stamp
Feature, Target

words or sentence? -> libraries to calculate sentiment
predictprob function
lower - case

In [ ]:
import pandas as pd
import datetime 

In [ ]:
#import the stock price data
SP_df = pd.read_csv("WIKI_PRICES.csv")

In [ ]:
SP_df.head()

In [ ]:
#check the number of data
len(SP_df)

In [ ]:
#import the news data
NEWS_df = pd.read_csv("uci-news-aggregator.csv")

In [ ]:
NEWS_df.head()

In [ ]:
#check the number of data
len(NEWS_df)

In [ ]:
companies = {"3M": "MMM",
             "American Express":"AXP",
             "Apple":"AAPL",
             "Boeing":"BA",
             "Caterpillar":"CAT",
             "Chevron":"CVX",
             "Cisco Systems":"CSCO",
             "Coca-Cola":"KO",
             "DuPont":"DD",
             "ExxonMobil":"XOM",
             "General Electric":"GE",
             "Goldman Sachs":"GS",
             "Home Depot":"HD",
             "IBM":"IBM",
             "Intel":"INTC",
             "Johnson & Johnson":"JNJ",
             "JPMorgan Chase":"JPM",
             "McDonald's":"MCD",
             "Merck":"MRK",
             "Microsoft":"MSFT",
             "Nike":"NKE",
             "Pfizer":"PFE",
             "Procter & Gamble":"PG",
             "Travelers":"TRV",
             "UnitedHealth Group":"UNH",
             "United Technologies":"UTX",
             "Verizon":"VZ",
             "Visa":"V",
             "Walmart":"WMT",
             "Walt Disney":"DIS"
}

In [ ]:
#try to check whether there is enough data for above companies
def count_news(companies):
    total = 0
    news_count = {}
    for key, item in companies.items():
        num =  len(NEWS_df[NEWS_df['TITLE'].str.contains(key, na=False)])
        total += num
        news_count[key] = num
    
    return total, news_count

total, news_count = count_news(companies)
print(total)
print(news_count)

In [ ]:
#try to check whether there is corresponding stock price data for above companies
for key, item in companies.items():
    if item in SP_df['ticker'].values:
        continue
    else:
        print('Following company doesn\'nt exist: {}'.format(item))

In [ ]:
#delete the companies with the small number of news in NEWS_df
for key, item in news_count.items():
    if news_count[key] < 100:
        companies.pop(key)

total_v2, news_count_v2 = count_news(companies)
print(total_v2)
print(news_count_v2)

In [ ]:
#drop the unnecessary column of NEWS_df
NEWS_df = NEWS_df.drop(["CATEGORY","STORY","HOSTNAME"], axis=1)
NEWS_df.columns

In [ ]:
#convert timestamp to date
NEWS_df['TIMESTAMP'] = pd.to_datetime(NEWS_df['TIMESTAMP'], unit='ms')

In [ ]:
#Create a function that determine whether the news title includes any company

def company_in_news(paragraph, companies):
    """
    input; paragraph: string, companies: dictionary
    output; boolean
    """
    words = paragraph.split()
    is_inNews = False
    for i in range(len(words)):
        if words[i] in companies.keys():
            is_inNews = True
    
    return is_inNews



memo for myself  
__number difference with str.contain()__  
len(NEWS_companies_df)

In [ ]:
#create the new dataframe which consists of relevant news only
NEWS_companies_df = NEWS_df[NEWS_df['TITLE'].apply(company_in_news, args=(companies,)) == True]

In [ ]:
#create a function which give the label of the company's ticker to the news

def label_news(paragraph, companies):
    """
    input; paragraph: string, companies: dictionary
    output; name: company's ticker
    """
    words = paragraph.split()
    for i in range(len(words)):
        if words[i] in companies.keys():
            return companies[words[i]]
    
    return 'NaN'

NEWS_companies_df['ticker'] = NEWS_companies_df['TITLE'].apply(label_news, args=(companies,))


memo for myself  
__could be together with company_in_news__  
__got a warning__

In [ ]:
#create a function that takes the corresponding the stock price move
#this function runs very slow. next cell might take more than 10 mins to run
def get_move(row, SP_df):
    
    """
    input; row: entire row, SP_df: DataFrame of stock price
    output; boolean
    """    
    
    date_tmp = row['TIMESTAMP']
    
    #check the hour of the news, and then if it's after 17, make the date next day.
    if date_tmp.hour >= 17:
        date = date_tmp + datetime.timedelta(days=1)
    else:
        date = date_tmp
    date1 = datetime.datetime.strftime(date, "%Y-%m-%d")

    #get the day which is one day before input date
    def get_day_before(date, SP_df):
        date_set = set(SP_df['date'])
        date_list = list(date_set)
        date_list.sort()

        ind = date_list.index(date)

        return date_list[ind-1]
    
    #get the date0
    date0 = get_day_before(date1, SP_df)
    
    day0_price = SP_df.loc[(SP_df['date'] == date0) & (SP_df['ticker'] == row['ticker']), 'adj_close']
    day1_price = SP_df.loc[(SP_df['date'] == date1) & (SP_df['ticker'] == row['ticker']), 'adj_close']
    
    #return True if the stock price went up
    if day1_price.iloc[0] > day0_price.iloc[0]:
        return True
    else:
        return False

In [ ]:
NEWS_companies_df['IsUp'] = NEWS_companies_df.apply(get_move, axis = 1, args=(SP_df,))